In [ ]:
import pandas as pd
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt

from scipy import signal as sig
from sklearn import preprocessing
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from numpy import fft

In [ ]:
#initial constants
delta_t = 0.00025

nb_ptw = 2.048 #seconds

#voltage
V = 3.7

ptw_disc = int(nb_ptw/delta_t) #timesteps
msg_interval = 40000 #timesteps

to_uWh = (1000*1000*delta_t*V)/3600


nb_active_energy = 30 #uWh
ltem_active_energy = 100 #uWh

In [ ]:
##CoAP 
coap_byte_step = 41

sweep_file1 = "../measurements/otii/sweep_coap_psm_test01.csv"
sweep_file2 = "../measurements/otii/sweep_coap_psm_test02.csv" 
sweep_file3 = "../measurements/otii/sweep_coap_psm_test03.csv" 


coap_nb_data = []
coap_nb_data.append(pd.read_csv(sweep_file1))
coap_nb_data.append(pd.read_csv(sweep_file2))
coap_nb_data.append(pd.read_csv(sweep_file3))


sweep_file1 = "../measurements/otii/sweep_ltem_coap_psm_test01.csv"
sweep_file2 = "../measurements/otii/sweep_ltem_coap_psm_test02.csv" 
sweep_file3 = "../measurements/otii/sweep_ltem_coap_psm_test03.csv"
sweep_file4 = "../measurements/otii/sweep_ltem_coap_psm_test04.csv"


coap_ltem_data = []
coap_ltem_data.append(pd.read_csv(sweep_file1))
coap_ltem_data.append(pd.read_csv(sweep_file2))
coap_ltem_data.append(pd.read_csv(sweep_file3))
coap_ltem_data.append(pd.read_csv(sweep_file4))


In [ ]:
##MQTT
mqtt_byte_step = 64

sweep_file1 = "../measurements/otii/sweep_mqtt_psm_test01.csv"
sweep_file2 = "../measurements/otii/sweep_mqtt_psm_test02.csv" 
#sweep_file3 = "../measurements/otii/sweep_mqtt_psm_test03.csv" 


mqtt_data = []
mqtt_data.append(pd.read_csv(sweep_file1))
mqtt_data.append(pd.read_csv(sweep_file2))

sweep_file1 = "../measurements/otii/sweep_ltem_mqtt_psm_test01.csv"
sweep_file2 = "../measurements/otii/sweep_ltem_mqtt_psm_test02.csv"
sweep_file3 = "../measurements/otii/sweep_ltem_mqtt_psm_test03.csv"

mqtt_ltem_data = []
mqtt_ltem_data.append(pd.read_csv(sweep_file1))
mqtt_ltem_data.append(pd.read_csv(sweep_file2))
mqtt_ltem_data.append(pd.read_csv(sweep_file3))



In [ ]:
coap_nb_energy = []
coap_nb_current = []
for i in coap_nb_data:
    coap_nb_energy.append(i["Arc Main Energy (J)"])
    coap_nb_current.append(i["Arc Main Current (A)"])

mqtt_nb_energy = []
mqtt_nb_current = []
for i in mqtt_data:
    mqtt_nb_energy.append(i["Arc Main Energy (J)"])
    mqtt_nb_current.append(i["Arc Main Current (A)"])

coap_ltem_energy = []
coap_ltem_current = []
for i in coap_ltem_data:
    coap_ltem_energy.append(i["Arc Main Energy (J)"])
    coap_ltem_current.append(i["Arc Main Current (A)"])

mqtt_ltem_energy = []
mqtt_ltem_current = []
for i in mqtt_ltem_data:
    mqtt_ltem_energy.append(i["Arc Main Energy (J)"])
    mqtt_ltem_current.append(i["Arc Main Current (A)"])

In [ ]:
plt.figure()
for i in coap_nb_energy:
    i.plot()

for i in coap_ltem_energy:
    i.plot()

for i in mqtt_nb_energy:
    i.plot()

for i in mqtt_ltem_energy:
    i.plot()   

In [ ]:
segments_coap_nb = []

idx = 0
indexes_coap_nb = []
for batch in coap_nb_current:
    print("Batch: " + str(idx))
    curr_set = []
    i = 160000 
    index_list = []
    endpoint = (batch.index.size)
    while(i < endpoint):
        curr_meas = batch.iloc[i]
        if(curr_meas > 0.04):    
            curr_set.append(batch.iloc[(i+4000):(i+120000)])
            i+=120000
            index_list.append(i)
        else:
            i += 1
    indexes_coap_nb.append(index_list)
    segments_coap_nb.append(curr_set)
    idx += 1

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)

idx_seg = 0

coap_nb_sums = np.zeros((len(segments_coap_nb[0]), len(segments_coap_nb)))

bytes_coap = range(0,coap_byte_step*len(segments_coap_nb[0]), coap_byte_step)

for i in segments_coap_nb:
    sum_acc = []
    idx_sample = 0
    for j in i:
        curr_sum = j.sum()*to_uWh 
        sum_acc.append(curr_sum)
        coap_nb_sums[idx_sample][idx_seg] = curr_sum
        idx_sample += 1
    
    print(np.size(sum_acc))
    plt.plot(bytes_coap,sum_acc, label = str(idx_seg))
    
    idx_seg += 1

#for i in range(len(bytes_coap)):
#    ax.annotate('(%d)' % (bytes_coap[i]), xy=(i*coap_byte_step,sum_acc[i]))

    
plt.grid()
    
plt.ylabel("Power consumption [uWh]")
plt.xlabel("Payload size [Bytes]")

In [ ]:
coap_nb_stds = []
coap_nb_avg = []

for i in coap_nb_sums:
    coap_nb_stds.append(np.std(i))
    coap_nb_avg.append(np.mean(i))
    
np.mean(coap_nb_stds)

In [ ]:
segments_coap_ltem = []

idx = 0
indexes_coap_ltem = []
for batch in coap_ltem_current:
    print("Batch: " + str(idx))
    curr_set = []
    i = 100000 
    index_list = []
    endpoint = (batch.index.size)
    while(i < endpoint):
        curr_meas = batch.iloc[i]
        if(curr_meas > 0.05):    
            curr_set.append(batch.iloc[(i+500):(i+60000)])
            i+=60000
            index_list.append(i)
        else:
            i += 1
    indexes_coap_ltem.append(index_list)
    segments_coap_ltem.append(curr_set)
    idx += 1

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)

idx_seg = 0

coap_ltem_sums = np.zeros((len(segments_coap_ltem[0]), len(segments_coap_ltem)))

for i in segments_coap_ltem:
    sum_acc = []
    idx_sample = 0
    for j in i:
        curr_sum = j.sum()*to_uWh 
        sum_acc.append(curr_sum)
        coap_ltem_sums[idx_sample][idx_seg] = curr_sum
        idx_sample += 1
    
    print(np.size(sum_acc))
    plt.plot(sum_acc, label = str(idx_seg))
    
    idx_seg += 1

#for i in range(len(bytes_coap)):
#    ax.annotate('(%d)' % (bytes_coap[i]), xy=(i*coap_byte_step,sum_acc[i]))

    
plt.grid()
    
plt.ylabel("Power consumption [uWh]")
plt.xlabel("Payload size [Bytes]")

In [ ]:
coap_ltem_stds = []
coap_ltem_avg = []

for i in coap_ltem_sums:
    coap_ltem_stds.append(np.std(i))
    coap_ltem_avg.append(np.mean(i))
    
np.mean(coap_ltem_stds)

In [ ]:
segments_mqtt_nb = []

idx = 0
indexes_mqtt_nb = []
for batch in mqtt_nb_current:
    print("Batch: " + str(idx))
    curr_set = []
    i = 200000 
    index_list = []
    endpoint = (batch.index.size)
    while(i < endpoint):
        curr_meas = batch.iloc[i]
        if(curr_meas > 0.04):    
            curr_set.append(batch.iloc[(i+4000):(i+120000)])
            i+=160000
            index_list.append(i)
        else:
            i += 1
    indexes_mqtt_nb.append(index_list)
    segments_mqtt_nb.append(curr_set)
    idx += 1

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)

idx_seg = 0

mqtt_nb_sums = np.zeros((len(segments_mqtt_nb[0]), len(segments_mqtt_nb)))
bytes_mqtt = range(0,mqtt_byte_step*len(segments_mqtt_nb[0]), mqtt_byte_step)

for i in segments_mqtt_nb:
    sum_acc = []
    idx_sample = 0
    for j in i:
        curr_sum = j.sum()*to_uWh 
        sum_acc.append(curr_sum)
        mqtt_nb_sums[idx_sample][idx_seg] = curr_sum
        idx_sample += 1
    
    print(np.size(sum_acc))
    plt.plot(bytes_mqtt,sum_acc, label = str(idx_seg))
    
    idx_seg += 1

#for i in range(len(bytes_coap)):
#    ax.annotate('(%d)' % (bytes_coap[i]), xy=(i*coap_byte_step,sum_acc[i]))

    
plt.grid()
    
plt.ylabel("Power consumption [uWh]")
plt.xlabel("Payload size [Bytes]")

In [ ]:
mqtt_nb_stds = []
mqtt_nb_avg = []

for i in mqtt_nb_sums:
    mqtt_nb_stds.append(np.std(i))
    mqtt_nb_avg.append(np.mean(i))
    
np.mean(mqtt_nb_stds)

In [ ]:
segments_mqtt_ltem = []

idx = 0
indexes_mqtt_ltem = []
for batch in mqtt_ltem_current:
    print("Batch: " + str(idx))
    curr_set = []
    i = 120000 
    index_list = []
    endpoint = (batch.index.size)
    while(i < endpoint):
        curr_meas = batch.iloc[i]
        if(curr_meas > 0.04):    
            curr_set.append(batch.iloc[(i+500):(i+100000)])
            i+=100000
            index_list.append(i)
        else:
            i += 1
    indexes_mqtt_ltem.append(index_list)
    segments_mqtt_ltem.append(curr_set)
    idx += 1

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)

idx_seg = 0

mqtt_ltem_sums = np.zeros((len(segments_mqtt_ltem[0]), len(segments_mqtt_ltem)))
bytes_mqtt = range(0,mqtt_byte_step*len(segments_mqtt_ltem[0]), mqtt_byte_step)

for i in segments_mqtt_ltem:
    sum_acc = []
    idx_sample = 0
    for j in i:
        curr_sum = j.sum()*to_uWh 
        sum_acc.append(curr_sum)
        mqtt_ltem_sums[idx_sample][idx_seg] = curr_sum
        idx_sample += 1
    
    print(np.size(sum_acc))
    plt.plot(bytes_mqtt,sum_acc, label = str(idx_seg))
    
    idx_seg += 1

#for i in range(len(bytes_coap)):
#    ax.annotate('(%d)' % (bytes_coap[i]), xy=(i*coap_byte_step,sum_acc[i]))

    
plt.grid()
    
plt.ylabel("Power consumption [uWh]")
plt.xlabel("Payload size [Bytes]")

In [ ]:
mqtt_ltem_stds = []
mqtt_ltem_avg = []

for i in mqtt_ltem_sums:
    mqtt_ltem_stds.append(np.std(i))
    mqtt_ltem_avg.append(np.mean(i))
    
np.mean(mqtt_ltem_stds)

In [ ]:
coap_ltem_smooth = sig.savgol_filter(coap_ltem_avg,9,3)
coap_nb_smooth = sig.savgol_filter(coap_nb_avg,9,3)
mqtt_nb_smooth = sig.savgol_filter(mqtt_nb_avg,9,3)
mqtt_ltem_smooth = sig.savgol_filter(mqtt_ltem_avg,9,3)

In [ ]:
fig, (ax1, ax2) = plt.subplots(1,2)

ax1.set_title('CoAP')
ax2.set_title('MQTT')

ax1.set(ylabel="Power consumption [uWh]")
ax1.set_xlabel("Payload size [Bytes]")
ax2.set_xlabel("Payload size [Bytes]")

#plt.plot(bytes_coap, coap_nb_avg   ,     label = "CoAP NB-IoT average")
ax1.plot(bytes_coap, coap_nb_smooth-nb_active_energy, "--" ,label = "NB-IoT")


#plt.plot(bytes_mqtt, mqtt_nb_avg   ,       label = "MQTT 2300 average")
ax2.plot(bytes_mqtt, mqtt_nb_smooth-nb_active_energy, "--" ,label = "NB-IoT")

#plt.plot(bytes_coap, coap_ltem_avg   ,     label = "CoAP LTE-M average")
ax1.plot(bytes_coap, coap_ltem_smooth-ltem_active_energy, "--", label = "LTE-M")

#plt.plot(bytes_mqtt, mqtt_ltem_avg   ,       label = "MQTT 2300 average")
ax2.plot(bytes_mqtt, mqtt_ltem_smooth-ltem_active_energy, "--" ,label = "LTE-M")
ax2.legend(loc='center left', bbox_to_anchor=(1, 0.5))

In [ ]:
coap_nb_avg[0]-30

In [ ]:
X = np.reshape(bytes_coap, (-1,1))
y = np.reshape(coap_nb_avg, (-1,1))

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [ ]:
regressor = LinearRegression()
regressor.fit(X_train, y_train)

intercept = regressor.intercept_[0]
coef = regressor.coef_[0]

In [ ]:
y_pred = regressor.predict(X_test)



In [ ]:
plt.figure()
plt.plot(X_test,y_pred.flatten())
#plt.plot(y_test.flatten())
#plt.plot(y_train.flatten())
plt.plot(bytes_coap, coap_nb_avg)

x_line = np.linspace(0,1440,100)
y_line = coef*x+intercept

plt.plot(x_line, y_line)



In [ ]:
#encapsulation calculation
#https://baturin.org/tools/encapcalc/